In [0]:
%sql
CREATE OR REPLACE TABLE formula_1.dim_meetings (
  meeting_key BIGINT PRIMARY KEY NOT NULL,
  meeting_name VARCHAR(100) NOT NULL,
  meeting_official_name VARCHAR(100),
  meeting_code CHAR(6),
  country_name VARCHAR(50), 
  location VARCHAR(50), 
  circuit_short_name VARCHAR(100), 
  meeting_date_start DATE
)

In [0]:
%sql
CREATE OR REPLACE TABLE formula_1.dim_sessions (
  session_key BIGINT PRIMARY KEY NOT NULL,
  session_name VARCHAR(100) NOT NULL,
  session_type VARCHAR(50) NOT NULL, 
  session_date_start DATE, 
  session_date_end DATE, 
  gmt_offset VARCHAR(50) NOT NULL
)

In [0]:
%sql
CREATE OR REPLACE TABLE formula_1.dim_drivers (
  driver_key BIGINT PRIMARY KEY NOT NULL GENERATED ALWAYS AS IDENTITY,
  first_name VARCHAR(50) NOT NULL, 
  last_name VARCHAR(50) NOT NULL, 
  full_name VARCHAR(50),
  broadcast_name VARCHAR(50), 
  driver_country_code CHAR(6), 
  team_name VARCHAR(50)
)

In [0]:
%sql
CREATE OR REPLACE TABLE formula_1.fact_laps_performance (
  id BIGINT PRIMARY KEY NOT NULL GENERATED ALWAYS AS IDENTITY,
  meeting_key BIGINT NOT NULL,
  session_key BIGINT NOT NULL,
  driver_key BIGINT NOT NULL,
  lap_number INT,
  lap_date_start DATE,
  duration_sector_1 DOUBLE,
  duration_sector_2 DOUBLE,
  duration_sector_3 DOUBLE,
  i1_speed DOUBLE,
  i2_speed DOUBLE,
  is_pit_out_lap BOOLEAN,
  lap_duration DOUBLE,
  segments_sector_1 ARRAY<INT>,
  segments_sector_2 ARRAY<INT>,
  segments_sector_3 ARRAY<INT>,
  st_speed DOUBLE,
  CONSTRAINT fk_meeting FOREIGN KEY (meeting_key)
  REFERENCES formula_1.dim_meetings (meeting_key)
  NOT ENFORCED,
  CONSTRAINT fk_session FOREIGN KEY (session_key)
  REFERENCES formula_1.dim_sessions (session_key)
  NOT ENFORCED,
  CONSTRAINT fk_driver FOREIGN KEY (driver_key)
  REFERENCES formula_1.dim_drivers (driver_key)
  NOT ENFORCED
)

In [0]:
%sql
INSERT INTO
  formula_1.dim_meetings
SELECT
  DISTINCT
    meeting_key,
    meeting_name,
    meeting_official_name, 
    meeting_code,
    country_name, 
    location, 
    circuit_short_name, 
    meeting_date_start
FROM
  formula_1.int_transform_laps_data;

In [0]:
%sql
SELECT COUNT(*) FROM formula_1.raw_meetings

In [0]:
%sql
SELECT COUNT(*) FROM formula_1.dim_meetings

In [0]:
%sql
INSERT INTO
  formula_1.dim_sessions
SELECT
  DISTINCT
    session_key,
    session_name,
    session_type, 
    session_date_start, 
    session_date_end, 
    gmt_offset
FROM
  formula_1.int_transform_laps_data;

In [0]:
%sql
SELECT COUNT(*) FROM formula_1.raw_sessions

In [0]:
%sql
SELECT COUNT(*) FROM formula_1.dim_sessions

In [0]:
%sql
INSERT INTO formula_1.dim_drivers (
  first_name, 
  last_name,
  full_name,
  broadcast_name, 
  driver_country_code, 
  team_name
)
SELECT DISTINCT
  first_name, 
  last_name,
  full_name,
  broadcast_name, 
  driver_country_code, 
  team_name
FROM formula_1.int_transform_laps_data;

In [0]:
%sql
SELECT 
  COUNT(*) 
FROM (
  SELECT 
    DISTINCT
      first_name, 
      last_name, 
      broadcast_name, 
      country_code, 
      team_name
  FROM
    formula_1.raw_drivers
)

In [0]:
%sql
SELECT COUNT(*) FROM formula_1.dim_drivers

In [0]:
%sql
SELECT COUNT(*) FROM formula_1.int_transform_laps_data

In [0]:
%sql
INSERT INTO formula_1.fact_laps_performance (
  meeting_key,
  session_key,
  driver_key,
  lap_number,
  lap_date_start,
  duration_sector_1,
  duration_sector_2,
  duration_sector_3,
  i1_speed,
  i2_speed,
  is_pit_out_lap,
  lap_duration,
  segments_sector_1,
  segments_sector_2,
  segments_sector_3,
  st_speed
)
SELECT
  DISTINCT
  meet.meeting_key,
  ses.session_key,
  driv.driver_key,
  lap_number,
  lap_date_start,
  duration_sector_1,
  duration_sector_2,
  duration_sector_3,
  i1_speed,
  i2_speed,
  is_pit_out_lap,
  lap_duration,
  segments_sector_1,
  segments_sector_2,
  segments_sector_3,
  st_speed
FROM 
  formula_1.int_transform_laps_data as laps
JOIN
  formula_1.dim_meetings as meet ON laps.meeting_key = meet.meeting_key
JOIN
  formula_1.dim_sessions as ses ON laps.session_key = ses.session_key
JOIN
  formula_1.dim_drivers as driv ON laps.driver_number = driv.driver_key

In [0]:
%sql
SELECT
  COUNT(*)
FROM (
  SELECT
  DISTINCT
  meet.meeting_key,
  ses.session_key,
  driv.driver_key,
  lap_number,
  lap_date_start,
  duration_sector_1,
  duration_sector_2,
  duration_sector_3,
  i1_speed,
  i2_speed,
  is_pit_out_lap,
  lap_duration,
  segments_sector_1,
  segments_sector_2,
  segments_sector_3,
  st_speed
FROM 
  formula_1.int_transform_laps_data as laps
JOIN
  formula_1.dim_meetings as meet ON laps.meeting_key = meet.meeting_key
JOIN
  formula_1.dim_sessions as ses ON laps.session_key = ses.session_key
JOIN
  formula_1.dim_drivers as driv ON laps.driver_number = driv.driver_key
)